In [1]:
# Demo app created by Arthur Jeannerot - November 2022
import bql
import pandas as pd
import ipywidgets as ipw
import plotly.graph_objects as go
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
bq = bql.Service()

In [3]:
class App(ipw.Tab):
    
    
    def __init__(self, bq = None):
        
        
        super().__init__()
        self.bq = bq
        self.widgets = {}
        self._build_view()
        self.chart_layout = {'template': 'plotly_dark',
                             'plot_bgcolor': 'rgba(33,33,33,33)',
                             'paper_bgcolor': 'rgba(33,33,33,33)'}
        
        
    def _build_view(self):
        
        # Labels
        self.widgets['ticker_lbl'] = ipw.Label(value = 'Ticker', layout = {'width': '70px'})
        self.widgets['oi_lbl'] = ipw.Label(value = 'Open Int. > ', layout = {'width': '70px'})
        self.widgets['start_lbl'] = ipw.Label(value = 'Start Date', layout = {'width': '70px'})
        self.widgets['end_lbl'] = ipw.Label(value = 'End Date', layout = {'width': '70px'})
        
        # Input Widgets
        self.widgets['ticker'] = ipw.Text(value = 'FJSA Comdty')
        self.widgets['oi'] = ipw.Text(value = '5')
        self.widgets['start_dt'] = ipw.DatePicker(value = datetime.date.today() - relativedelta(days = 9))
        self.widgets['end_dt'] = ipw.DatePicker(value = datetime.date.today() - relativedelta(days = 2))
        
        
        # Label + Widget HBox
        self.widgets['ticker_ui'] = ipw.HBox([self.widgets['ticker_lbl'], self.widgets['ticker']])
        self.widgets['oi_ui'] = ipw.HBox([self.widgets['oi_lbl'], self.widgets['oi']])
        self.widgets['start_ui'] = ipw.HBox([self.widgets['start_lbl'], self.widgets['start_dt']])
        self.widgets['end_ui'] = ipw.HBox([self.widgets['end_lbl'], self.widgets['end_dt']])
        
        
        # Button
        self.widgets['btn'] = ipw.Button(description = 'Get Data')
        self.widgets['btn'].button_style = 'Primary'
        self.widgets['btn'].on_click(self.controller)

        
        # Controls
        self.widgets['controls'] = ipw.VBox([self.widgets['ticker_ui'],
                                             self.widgets['oi_ui'],
                                             self.widgets['start_ui'],
                                             self.widgets['end_ui'],
                                             self.widgets['btn']])
        
        self.children = [self.widgets['controls']]
        self.set_title(0, 'Options Summary')
        
        
    def read_ui(self):
        
        ui = {'ticker': self.widgets['ticker'].value,
              'oi': self.widgets['oi'].value,
              'start': self.widgets['start_dt'].value,
              'end': self.widgets['end_dt'].value}
        
        return ui
    
    
    def get_oi_data(self, ui):
        
        
        oi = self.bq.data.open_int()
        
        univ = self.bq.univ.futures(ui['ticker']).options().filter(oi > ui['oi'])
        fld = {'Open Int': oi.group(self.bq.data.strike_px()).sum()}
        
        req = bql.Request(univ, fld, with_params = {'mode': 'cached'})
        res = self.bq.execute(req)
        
        df = res[0].df()
        df = df.sort_values(by = df.columns[-2], ascending = True)
        
        return df
    
    
    def get_volume_chg(self, ui):
    
    
        vol_chg = self.bq.data.px_volume(fill = 'prev', dates = self.bq.func.range(ui['start'], ui['end'])).net_chg().dropna(True)
        univ = self.bq.univ.futures(ui['ticker']).options()

        top25 = vol_chg.group().sort(order = 'desc').first(25).ungroup(ungrouporder = 'current')
        bottom25 = vol_chg.group().sort(order = 'asc').first(25).ungroup(ungrouporder = 'current')

        flds = {'top25': top25, 'bottom25': bottom25}

        req = bql.Request(univ, flds, with_params = {'mode': 'cached'})
        res = self.bq.execute(req)

        df_top = res[0].df()
        df_btm = res[1].df()

        return df_top, df_btm
    
    
    def get_oi_chg(self, ui):
    
    
        vol_chg = self.bq.data.open_int(fill = 'prev', dates = self.bq.func.range(ui['start'], ui['end'])).net_chg().dropna(True)
        univ = self.bq.univ.futures(ui['ticker']).options()

        top25 = vol_chg.group().sort(order = 'desc').first(25).ungroup(ungrouporder = 'current')
        bottom25 = vol_chg.group().sort(order = 'asc').first(25).ungroup(ungrouporder = 'current')

        flds = {'top25': top25, 'bottom25': bottom25}

        req = bql.Request(univ, flds, with_params = {'mode': 'cached'})
        res = self.bq.execute(req)

        df_top = res[0].df()
        df_btm = res[1].df()

        return df_top, df_btm
    
    
    def replace_opt_id(self, df):
        
        
        flds = {'tenor': self.bq.data.fut_month_yr(),
                'put_call': self.bq.data.put_call(),
                'strike': self.bq.data.strike_px()}
                        
        req = bql.Request(self.bq.univ.list(list(df.index)), flds)
        res = self.bq.execute(req)

        data = pd.concat([fld.df()[fld.name] for fld in res], axis = 1)
        data['des'] = data['tenor'].astype(str) + ' ' + data['strike'].astype(str) + ' ' + data['put_call'].astype(str)

        df = pd.concat([df, data], axis = 1)
        df = df.set_index('des')

        return df            
    
        
    def chart_oi(self, df):
        
        
        traces = go.Bar(x = df.index, y = df['Open Int'])
        fig = go.FigureWidget(data = traces, layout = self.chart_layout)
        
        fig.update_layout(title = 'Open Interest by Strike Price', title_x = 0.5)
        
        
        return fig
    
    
    def chart_vol_chg(self, dfs):
        
        
        top25 = go.Bar(x = dfs[0].index, y = dfs[0]['top25'])
        bottom25 = go.Bar(x = dfs[1].index, y = dfs[1]['bottom25'])
        
        top25_fig = go.FigureWidget(data = top25, layout = self.chart_layout)
        top25_fig.update_layout(title = 'Top 25 Volume Increases', title_x = 0.5)
        top25_fig.update_xaxes(tickangle = 45)
        bottom25_fig = go.FigureWidget(data = bottom25, layout = self.chart_layout)
        bottom25_fig.update_layout(title = 'Top 25 Volume Decreases', title_x = 0.5)
        bottom25_fig.update_xaxes(tickangle = 45)
        
        charts = ipw.HBox([top25_fig, bottom25_fig])
        
        return charts
    
    
    def chart_oi_chg(self, dfs):

        
        top25 = go.Bar(x = dfs[0].index, y = dfs[0]['top25'])
        bottom25 = go.Bar(x = dfs[1].index, y = dfs[1]['bottom25'])
        
        top25_fig = go.FigureWidget(data = top25, layout = self.chart_layout)
        top25_fig.update_layout(title = 'Top 25 Open Int. Increases', title_x = 0.5)
        top25_fig.update_xaxes(tickangle = 45)
        bottom25_fig = go.FigureWidget(data = bottom25, layout = self.chart_layout)
        bottom25_fig.update_layout(title = 'Top 25 Open Int. Decreases', title_x = 0.5)
        bottom25_fig.update_xaxes(tickangle = 45)
        
        charts = ipw.HBox([top25_fig, bottom25_fig])
        
        return charts
    
    
    def set_error_msg(self,error):
        
        err_widget = ipw.HTML(f'<p style="color:red;" >{error}</p>')
        
        self.children = [ipw.VBox([self.widgets['controls'], err_widget])]
        
        
    def controller(self, btn_click):
        
        self.widgets['btn'].disabled = True
        self.widgets['btn'].description = 'Requesting Data...'
        self.widgets['btn'].button_style = 'warning'
        
        try:
            self.children = [self.widgets['controls']] # Clear any previous output
            ui = self.read_ui() # Read user inputs

            oi_data = self.get_oi_data(ui) # Pull OI strike data with user inputs
            vol_data = self.get_volume_chg(ui) # Pull volume change data with user inputs            
            vol_dfs = [self.replace_opt_id(vol_data[i]) for i in range(len(vol_data))] # Replace option ID's
            oi_chg_data = self.get_oi_chg(ui) # Pull OI change data with user inputs
            oi_dfs = [self.replace_opt_id(oi_chg_data[i]) for i in range(len(oi_chg_data))] # Replace option ID's

            # Create charts
            oi_chart = self.chart_oi(oi_data)
            vol_charts = self.chart_vol_chg(vol_dfs)
            oi_chg_charts = self.chart_oi_chg(oi_dfs)
            
            # Combined charts into a widget container
            charts = ipw.Accordion([oi_chart, vol_charts, oi_chg_charts])
            
            # Rename the chart containers
            titles = ['Open Interest by Strike Price', 'Volume Movers', 'Open Interest Movers']
            for i in range(3):
                charts.set_title(i, titles[i])         
            
            # Pass charts to app
            self.children = [ipw.VBox([self.widgets['controls'],
                                       charts,
                                      ])]
            
        except Exception as e:
            self.set_error_msg(str(e))
        
        self.widgets['btn'].disabled = False
        self.widgets['btn'].description = 'Get Data'
        self.widgets['btn'].button_style = 'Primary'
        
        

In [4]:
app = App(bq)

In [5]:
app

App(children=(VBox(children=(HBox(children=(Label(value='Ticker', layout=Layout(width='70px')), Text(value='FJ…

In [6]:
ui = app.read_ui()

In [ ]:
oi_df = app.get_oi_data(ui)

In [ ]:
oi_df

In [ ]:
oi_df.to_excel('export.xlsx')